In [ ]:
############ Import libraries

import pandas as pd
import numpy as np
from pandas import read_csv


import random
import imblearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids 


from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier
from sklearn import tree
import seaborn as sns
from sklearn.svm import SVC
from sklearn import svm
import lightgbm as lgb


# Data Pre-processing

In [ ]:
## Read Dataset

df = pd.read_table("Glycation.elm")
#print(df.head())
print(len(df))


## Table Split

table1= df.iloc[:,[0,2]]
table2 =df.iloc[:,[0,4]]


#print(table2.iloc[0,0])

#print(table1)

In [ ]:
## convert to csv file for table1

tb1=pd.DataFrame({'PLMD ID':table1.iloc[:,0],'Position':table1.iloc[:,1] })
filename= 'table1.csv'
tb1.to_csv(filename,index=False)


In [ ]:
## convert to csv file for table 2

tb2=pd.DataFrame({'PLMD ID':table2.iloc[:,0],'Sequence':table2.iloc[:,1] })
filename= 'table2.csv'
tb2.to_csv(filename,index=False)


In [ ]:
## Convert To fasta(File Write)

dataWrite = open("BeforeCD-HIT.fasta","w")
l = len(table2)
#print(l)
for i in range (l):
  dataWrite.write(">" + table2.iloc[i,0] + "\n" + table2.iloc[i,1]+"\n")

dataWrite.close()
#print(dataWrite.read())

In [ ]:
##### Convert fasta to CSV

dataRead = open('1593951336.fas.1','r')
v=[]
#print(dataRead.read())

for line in dataRead:
  v.append(line)


#print(len(v))
pid=[]
seq=[]
i=0

for i in range(len(v)):
  if(i%2==0):
    pid.append(v[i])

for i in range(len(v)):
  if(i%2!=0):
    seq.append(v[i])

#print(len(seq))
#print(pid)

In [ ]:
###  Remove ">" this sign from pid

x=len(pid)
#print(pid)
for i in range(x):
  t=pid[i]
  t=t.replace(">","")
  t=t.replace("\n","")
  pid[i]=t

#print(pid)

In [ ]:
###  Convert CSV file after CD-HIT

f=pd.DataFrame({'PLMD ID':pid,'Sequence':seq })
filename= 'afterCD-HIT.csv'
f.to_csv(filename,index=False)

In [ ]:
##### For Finding All Peptides


out= pd.read_csv('afterCD-HIT.csv')

seq=out.iloc[:,1]

window_size=23                         ##### for window size
half=int(np.floor(window_size/2))      #### half of window size

print("*****************  Here Window Size is:",window_size)

s_len=len(seq)
#print("SEQ lenght: ",s_len)

all_pep=[]

c=0
for i in range(s_len):
  pep=seq[i]
  #pep.rstrip("\n")
  pep.replace("\n","")
  #print("Full Seq: ",pep)
  singleSeq_len=len(pep)
  #print("Single Seq Length: ",singleSeq_len)
  for j in range(singleSeq_len):
    if( pep[j] == "K" ):
      c+=1
      s=j-1         ### s mane start
      e=j+half       ### e mane end

      #print("Start, end,position,strLen: ",s,e,j,singleSeq_len)
      
      if( s>=half and e<(singleSeq_len-1) ):    ### Sob thik ase
        s_l=j-half
        f=pep[s_l:e+1]
        all_pep.append(f)
        #print("All Peptides: ",f)
        #if( len(f) < 23 ):
         #       print("Less 23")
        #else:
         # print("Length okay from sob thik")
      

      elif(s>=half and e>=(singleSeq_len-2)):
        f_len=len(pep)-1
        #print("Len: ",f_len)
        #print("********Right e problem")
        #print("SEQ: ",pep)
        s_l=j-half
        f=pep[s_l:f_len]
        #print("JHDHD: ",f,len(f))
        r=window_size-len(f)
        #print("Need: ",r)
        for h in range(r):
          f = f + "X"
        
        #print("After adding X: ",f)
        all_pep.append(f)
        #if( len(f) < 23 ):
          #      print("Less 23")
        #else:
         # print("Length okay from right ")


      elif( s<half and e<=singleSeq_len):
        f_len=len(pep)
        #print("******First e problem")
        #print("start, end, pos,len:",s,e,j,f_len)
        f=pep[0:e+1]
        #print("Before adding X: ",f)
        r=half-(s+1)
        #print("Need X: ",r)
        
        for q in range(r):
          f="X"+f
        #print("After adding X: ",f)
        all_pep.append(f)
        #if( len(f) < 23 ):
         #       print("Less 23")
        #else:
          #print("Length okay from first e")

      else:
        print("****Prolem ase ")

print("Count of K: ",c)

print(len(all_pep))

print(len(all_pep[100]))
print(all_pep[100])

In [ ]:
##### Remove X from all seq

remove_all_X=[]
l=len(all_pep[0])
print("All pep: ",len(all_pep))

for i in range(len(all_pep)):
  r=all_pep[i]
  #print(r)
  if r[0]== 'X' or r[window_size-1]== 'X':
    remove_all_X.append(r)

print("With X: ",len(remove_all_X))



### For Find all pep without X from All pep
after_all_X=[]
temp=all_pep[:]
count=0
for i in range(len(all_pep)):
  for j in range(len(remove_all_X)):
    if( all_pep[i] == remove_all_X[j] ):
      temp.pop(i-count)
      count+=1
      break
after_all_X=temp

print("Without X: ",len(after_all_X))

In [ ]:
### For Finding Positive Peptides

table1=pd.read_csv('table1.csv')

positive_seq=[]
positive_label=[]


seq_id=out.iloc[:,0]    #### out theke value niyar jonno
seq_s=out.iloc[:,1]
s_len=len(seq_id)



tid=table1.iloc[:,0]    #### table1 er value niyar jonno
tpos=table1.iloc[:,1]
t_len=len(tid)
#print(t_len)

u=0

for i in range(t_len):          #### for table 1
    a_id=tid[i]                      ### for store table1 er id
    a_pos=tpos[i]                       #### for store table1 er position
    #print("number: ",i)
    for j in range(s_len):
        if( a_id == seq_id[j] and a_pos< len(seq_s[j])):      ### For checking PLMD id
            start=a_pos-1-1
            end=a_pos+half-1
            t=seq_s[j]
            #print("SEQ: ",t)
            t=t.rstrip("\n")
            s=len(t)                ### s for akta seq er length
            
            #print("AFTER NEW LINE: ",t)
            #print("Seq: ",t)
            #print("Position:" ,a_pos)
            #print("Start: ",start)
            #print("End",end)
            #print("ID Match")

            if( start>= half and end <(s-1) ):      ### Sob jaigai e thik ase
              l1=a_pos-half-1
              w=t[l1:end+1]
              #print("Seq Sob thik: ",len(w))
             #if( w[11] == "K" ):
              #  print("Okay")
             # else:
              print("Problem of length at sob: ",w)
              positive_seq.append(w)
              positive_label.append(1)
              u+=1
            

            elif( start>=half and end>=(s-1)):
              l1=a_pos-half-1
              w=t[l1:s]
              s=len(seq_s[j])
              print("Right problem: ",w)
              r=half-(s-a_pos)+1
              for h in range(r):
                w = w + "X"

              #if( w[11] == "K" ):
               # print("Okay")
              #else:
              print("Problem of length at last e: ",w) 
              positive_seq.append(w)
              positive_label.append(1)
              #print("Seq last e problem:",w,len(w))
              u+=1

            
            elif(start<half and end<=s):    # First e problem
              #print("start, end, pos:",start,end,a_pos)
              w=t[0:end+1]
              #print(w)
              start=half-start
              for f in range(start-1):
                w="X" + w
              print("Seq first e problem: ",w)
             # if( w[11] == "K" ):
              #  print("Okay")
              #else:
               # print("Problem of length at first e")
              positive_seq.append(w)
              positive_label.append(1)
              u+=1



            else:
              print("Problem to find sequence, Len:******* ",s)
              print("Start, end", start, end)
              print(t,(a_pos))
              print(t[a_pos-1])



                
print("Total Match:",u)   


print(len(positive_seq[100]))
#print(positive_label)    
#print(len(positive_label))    


#print("Positive Peptides: ",len(positive_seq))


In [ ]:
##### For checking all positive peptides length is equal or not

print("Total Seq Before Check:",len(positive_seq))
c=0
for i in range( len(positive_seq) ):
  if( len(positive_seq[i]) == window_size):
    #print(positive_seq[i])
    #print(i)
    c+=1
    

print("Total Count:",c)

In [ ]:
############ Remove X from positive data


remove_positive_X=[]
l=len(positive_seq[0])
print('Length of positive data:',l)

print('Number of Positive data: ',len(positive_seq))


for i in range(len(positive_seq)):
  r=positive_seq[i]
  #print(r)
  if r[0]== 'X' or r[window_size-1]== 'X':
    remove_positive_X.append(r)

print('With X: ',len(remove_positive_X))


### For Find positive without X from All positive

after_positive_X=[]
temp1=positive_seq[:]
count1=0
for i in range(len(positive_seq)):
  for j in range(len(remove_positive_X)):
    if( positive_seq[i] == remove_positive_X[j] ):
      temp1.pop(i-count1)
      count1+=1
      break
after_positive_X=temp1

print("Without X: ",len(after_positive_X))


In [ ]:
### For Finding Negative from All peptides

print("All peptides: ",len(all_pep))

negative_seq=[]
temp2=all_pep[:]
count2=0
for i in range(len(all_pep)):
  for j in range(len(positive_seq)):
    if( all_pep[i] == positive_seq[j] ):
      temp2.pop(i-count2)
      count2+=1
      break
negative_seq=temp2

print("Negative: ",len(negative_seq))
print("Positive: ",len(positive_seq))

In [ ]:
### For Finding Negative without X from All without X

after_negative_X=[]
temp2=after_all_X[:]
count2=0
print("All: ",len(after_all_X))
for i in range(len(after_all_X)):
  for j in range(len(after_positive_X)):
    if( after_all_X[i] == after_positive_X[j] ):
      temp2.pop(i-count2)
      count2+=1
      break
after_negative_X=temp2

print("With X: ",count2)
print("Without X: ",len(after_negative_X))

In [ ]:
##### For checking all negative peptides length is equal or not


print("Total Seq Before Check:",len(after_negative_X))
c=0
for i in range( len(after_negative_X) ):
  if( len(after_negative_X[i]) == window_size):
    #print(after_negative_X[i])
    #print(i)
    c+=1
    

print("Total Count:",c)

In [ ]:
######## Make positive labels

positive_label=[]
for i in range(len(after_positive_X)):
    positive_label.append(1)

print("Positive Length: ",len(positive_label))


###convert to csv file for positive data

positive_data=pd.DataFrame({'Peptides':after_positive_X,'Labels':positive_label })
filename= 'positive_data.csv'
positive_data.to_csv(filename,index=False)




######## Make negative labels

negative_label=[]
for i in range(len(after_negative_X)):
    negative_label.append(0)

print("Negative Length: ",len(negative_label))


##### Convert to csv file for negative data

negative_data=pd.DataFrame({'Peptides':after_negative_X,'Labels':negative_label })
filename= 'negative_data.csv'
negative_data.to_csv(filename,index=False)


In [ ]:
## Convert To fasta for negative seq

dataWrite = open("negative_seq_23.fasta","w")
l = len(after_negative_X)


#print(l)
for i in range (l):
  dataWrite.write(">" + str(i) + "\n" + after_negative_X[i]+"\n")

dataWrite.close()
#print(dataWrite.read())


## Convert To fasta for positive seq

dataWrite = open("positive_seq_23.fasta","w")
l = len(after_positive_X)

i=0

#print(l)
for i in range (l):
  dataWrite.write(">" + str(i) + "\n" + after_positive_X[i]+"\n")

dataWrite.close()
#print(dataWrite.read())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#    AAI 

In [ ]:
##### For Positive data ----- Make AAIndex

encode=['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V']

positive_data= pd.read_csv('/content/drive/My Drive/Conference/positive_data.csv')
#print(positive_data)

positive_seq=positive_data.iloc[:,0]
seq_number=len(positive_seq)
seq_len=len(positive_seq[0])


print("Total Seq: ",seq_number)
print("Lenght of each sequence: ",seq_len)
print("Total Encoding: ",len(encode))

## Read Dataset
df =pd.read_csv("/content/drive/My Drive/Conference/AAIndex_28Features_Manual.csv")
t=df.iloc[:,1:]
print("Before transpose: ",t.shape)


aai_positive= np.zeros((seq_number,seq_len,t.shape[0]))
print("Length of feature vector: ",aai_positive.shape)
#print(f_positive)



tr=np.transpose(t)       ######## Make transpose matrix
#print(tr)
#print(tr.iloc[0,:])
#print(len(tr.iloc[0,:]))
print("After transpose: ",tr.shape)



for n in range(seq_number):
  x1=positive_seq[n]
  #print(x1)
  for i in range(seq_len):
    x=x1[i]
    #print(x)
    for j in range(len(encode)):
      if x==encode[j]:
        #print("Match")
        aai_positive[n,i,:]=tr.iloc[j,:]

print(aai_positive.shape)


#print(tr.shape[0])

In [ ]:
##### For Negative data ----- Make AAIndex


negative_data= pd.read_csv('/content/drive/My Drive/Conference/negative_data.csv')
#print(negative_data)
negative_seq=negative_data.iloc[:,0]


seq_number=len(negative_seq)

print("Total Seq: ",seq_number)
print("Lenght of each sequence: ",seq_len)
print("Total Encoding: ",len(encode))


aai_negative= np.zeros((seq_number,seq_len,len(tr.iloc[0,:])))
print("Length of feature vector: ",aai_negative.shape)
#print(f_negative)



for n in range(seq_number):
  x1=negative_seq.iloc[n]
  #print(x1)
  for i in range(seq_len):
    x=x1[i]
    #print(x)
    for j in range(len(encode)):
      if x==encode[j]:
        #print("Match")
        aai_negative[n,i,:]=tr.iloc[j,:]

print(aai_negative.shape)



In [ ]:
####### Positive data reshape for AAI


print("Before reshape: ",aai_positive.shape)
aai_positive_reshape=np.reshape(aai_positive,(aai_positive.shape[0],aai_positive.shape[1]*aai_positive.shape[2]))

print("After reshape: ",aai_positive_reshape.shape)



####### Negative data reshape for AAI


print("Before reshape: ",aai_negative.shape)
aai_negative_reshape=np.reshape(aai_negative,(aai_negative.shape[0],aai_negative.shape[1]*aai_negative.shape[2]))

print("After reshape: ",aai_negative_reshape.shape)



# CKSAAP 

In [ ]:
##### For Positive data ----- import CKSAAP where k=5

positive_cksaap= pd.read_csv('/content/drive/My Drive/Conference/cksaap_positive_23.csv')

print(positive_cksaap.shape)
positive_cksaap=positive_cksaap.iloc[:,1:]
print(positive_cksaap.shape)


In [ ]:
##### For Negative data ----- import CKSAAP where k=5

negative_cksaap= pd.read_csv('/content/drive/My Drive/Conference/cksaap_negative_23.csv')

print(negative_cksaap.shape)
negative_cksaap=negative_cksaap.iloc[:,1:]
print(negative_cksaap.shape)

# AAC 

In [ ]:
##### For Positive data ----- import AAC

positive_aac= pd.read_csv('/content/drive/My Drive/Conference/aac_positive_23.csv')

print(positive_aac.shape)
positive_aac=positive_aac.iloc[:,1:]
print(positive_aac.shape)


In [ ]:
##### For Negative data ----- import AAC

negative_aac= pd.read_csv('/content/drive/My Drive/Conference/aac_negative_23.csv')

print(negative_aac.shape)
negative_aac=negative_aac.iloc[:,1:]
print(negative_aac.shape)


#    Merge AAC, CKSAAP,  AAI for Positive

In [ ]:
print(positive_cksaap.shape)
print(positive_aac.shape)
print(aai_positive_reshape.shape)

a=positive_cksaap.shape[0]
b=positive_cksaap.shape[1] + positive_aac.shape[1] +  aai_positive_reshape.shape[1]

positive_feature=np.zeros((a,b))

positive_feature=np.append(positive_aac,positive_cksaap, axis=1)
print(positive_feature.shape)
positive_feature=np.append(positive_feature,aai_positive_reshape, axis=1)
print(positive_feature.shape)


#    Merge AAC, CKSAAP,  AAI for Negative

In [ ]:
print(negative_cksaap.shape)
print(negative_aac.shape)
print(aai_negative_reshape.shape)

a=negative_cksaap.shape[0]
b=negative_cksaap.shape[1] + negative_aac.shape[1] +  aai_negative_reshape.shape[1]

negative_feature=np.zeros((a,b))

negative_feature=np.append(negative_aac,negative_cksaap, axis=1)
print(negative_feature.shape)
negative_feature=np.append(negative_feature,aai_negative_reshape, axis=1)
print(negative_feature.shape)



In [ ]:
####### Feature Scaling

print(positive_feature.shape)
print(negative_feature.shape)


# define min max scaler
scaler = MinMaxScaler()

# transform data
positive_feature_minmax = scaler.fit_transform(positive_feature)

# transform data
negative_feature_minmax = scaler.fit_transform(negative_feature)

In [ ]:
###### Positive and negative data minmax

posi_train_minmax, posi_test_minmax= train_test_split(positive_feature_minmax, test_size=0.2)
print('Positive train: ',posi_train_minmax.shape)
print('Positive test: ',posi_test_minmax.shape)


nega_train_minmax, nega_test_minmax= train_test_split(negative_feature_minmax, test_size=0.2)
print('Negative train: ',nega_train_minmax.shape)
print('Negative test: ',nega_test_minmax.shape)



## **Merge Test Data(merge_feature_test, merge_y_test)**






In [ ]:
####### Merge Test Data(merge_feature_test, merge_y_test)

merge_feature_test=np.zeros((len(posi_test_minmax)+len(nega_test_minmax),posi_test_minmax.shape[1]))
print(merge_feature_test.shape)

merge_feature_test=np.concatenate((posi_test_minmax,nega_test_minmax))
print(merge_feature_test.shape)

##### label

merge_y_test=[]

for i in range(len(posi_test_minmax)):     #### for positive
    merge_y_test.append(1)

    
for j in range(len(nega_test_minmax)):    #### for negative
    merge_y_test.append(0)

print(len(merge_y_test))


#**Auto Encoder**

In [ ]:
########## Imbalance Solve By Autoecoder

from keras.models import Model
from keras.layers import  Input, Dense
import numpy as np
import matplotlib.pyplot as plt

print("Starting Auto Encoder.............")

In [ ]:
####### Variable Set

ratio=len(nega_train_minmax)/len(posi_train_minmax)
print("Negative To Positive Ratio: ",ratio)

input_size=posi_train_minmax.shape[0]
feature_size=posi_train_minmax.shape[1]
print("Positive Train Input Size: ",input_size)
print("Feature Size: ",feature_size)

In [ ]:
#### Autoencoder Model

encoding_dim=64

input_data=Input(shape=(feature_size,))
encoded=Dense(encoding_dim, activation='relu') (input_data)
decoded=Dense(feature_size,activation='sigmoid') (encoded)

autoencoder= Model(input_data, decoded)

encoder= Model(input_data, encoded)

encoded_input=Input(shape=(encoding_dim,))

decoder_layer= autoencoder.layers[-1]

decoder= Model(encoded_input, decoder_layer(encoded_input))


In [ ]:
### Compile and train the model

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(posi_train_minmax,posi_train_minmax,epochs=50, batch_size=256, shuffle=True)


In [ ]:
#### Predict with negative(majority class)

encoded_input=encoder.predict(nega_train_minmax)
decoded_input= decoder.predict(encoded_input)


In [ ]:
#### Predict with positive(minority class)

encoded_input_posi=encoder.predict(posi_train_minmax)
decoded_input_posi= decoder.predict(encoded_input)


In [ ]:
##### Calculate Distance For NUS-1
'''
import math

distance_list=[]

for i in range(nega_train_minmax.shape[0]):
  t=sum(pow(nega_train_minmax[i]-decoded_input[i],2))
  distance_list.append(t)

#print(distance_list)

sort_index=np.argsort(distance_list)[::-1] #### Decending Order Sort
#print(sort_index)

# Choose index according to sorting
fix_index=sort_index[0:input_size]
print(fix_index)
'''

In [ ]:
##### Calculate Distance for Majority class(Negative)

import math as m

distance_list_majority=[]

for i in range(nega_train_minmax.shape[0]):
  t=m.fsum(pow(nega_train_minmax[i]-decoded_input[i],2))
  distance_list_majority.append(t)

print(len(distance_list_majority))

sort_index_majority=np.argsort(distance_list_majority)[::-1] #### Decending Order Sort
#print(sort_index)


In [ ]:
##### Calculate Distance For Minority class(Positive)

import math

distance_list_minority=[]

for i in range(posi_train_minmax.shape[0]):
  t=m.fsum(pow(posi_train_minmax[i]-decoded_input_posi[i],2))
  distance_list_minority.append(t)

#print(distance_list)

sort_index_minority=np.argsort(distance_list_minority)[::-1] #### Decending Order Sort
#print(sort_index)

h=int(np.ceil(len(sort_index_minority)/2))
print(h)

s=0
for i in range(h):
  j=sort_index_minority[i]
  s=s+distance_list_minority[j]

last_mid_avrg=s/h
print("Last Mid Average:",last_mid_avrg)
e=sort_index_minority[0]
max_dis=distance_list_minority[e]
print("Max Distance:",max_dis)

In [ ]:
##### Select Index for Majority

select_index_majority=[]

for j in sort_index_majority:
  d=distance_list_majority[j]
  if d > max_dis or d > last_mid_avrg:
    select_index_majority.append(j)

print(len(select_index_majority))
  

In [ ]:
########### Select negative data

s=len(select_index_majority)
e=np.zeros((s,feature_size))
e=nega_train_minmax
final_nega_train=np.zeros((s,feature_size))

j=0
for i in select_index_majority:
  final_nega_train[j]=e[i]
  j=j+1
  
print("Final Negative Data Size:",final_nega_train.shape)

In [ ]:
print(final_nega_train.shape)
print(posi_train_minmax.shape)


In [ ]:
##### Error Find

print(select_index_majority[0])
a=final_nega_train[0]-nega_train_minmax[22886]
b=m.fsum(a)
print(b)

# Combine Positive & Negative  (merge_feature, merge_y)

In [ ]:
#   Merge Train Data

merge_feature=np.zeros((len(posi_train_minmax)+len(final_nega_train),posi_train_minmax.shape[1]))
print(merge_feature.shape)

merge_feature=np.concatenate((posi_train_minmax,final_nega_train))
print(merge_feature.shape)

##### label

merge_y=[]

for i in range(len(posi_train_minmax)):     #### for positive
    merge_y.append(1)

    
for j in range(len(final_nega_train)):    #### for negative
    merge_y.append(0)

print(len(merge_y))


In [ ]:
print('Original dataset shape {}'.format(Counter(merge_y)))


# Feature Selection (merge_feature_best, merge_y) 

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_regression

feature_selection_model=SelectKBest(score_func=f_regression,k=500)

merge_feature_best = feature_selection_model.fit_transform(merge_feature, merge_y)

print(merge_feature_best.shape)



In [ ]:
print('Original dataset shape {}'.format(Counter(merge_y)))

In [ ]:
### Feature Index Find For Test Set

select_feature_index_score=feature_selection_model.scores_
print(select_feature_index_score)

sort_index_feature=np.argsort(select_feature_index_score)[::-1] #### Decending Order Sort
print(sort_index_feature)

sort_index_feature=sort_index_feature[0:500]
print(len(sort_index_feature))


In [ ]:
#### Select the sorted feature for test set

f_test=np.zeros((12388,500))
print(f_test.shape)
print(len(sort_index_feature))

j=0
for i in sort_index_feature:
  f_test[:,j]=merge_feature_test[:,i]
  j=j+1

#print(f_test[0,:])


In [ ]:
############## Split positive & negative
"""
positive=np.zeros((3107,500))
print(positive.shape)
positive_y=[]

negative=np.zeros((58827,500))
print(negative.shape)
negative_y=[]
j=0
k=0

for i in range(len(merge_y)):
    if (merge_y[i]==1):
        t=merge_feature_best[i]
        positive[j,:] = t
        positive_y.append(1)
    elif (merge_y[i]==0):
        r=merge_feature_best[i]
        negative[k,:]  = r
        negative_y.append(0)

print(len(positive))

print(len(negative))

"""

In [ ]:
######## Split test and train
"""

positive_train_set,positive_test_independent,y_positive_train_set,y_positive_test_independent=train_test_split(positive, positive_y,test_size=0.2)

print(len(positive_train_set))
print(len(positive_test_independent))


negative_train_set,negative_test_independent,y_negative_train_set,y_negative_test_independent=train_test_split(negative, negative_y,test_size=0.010565)

print(len(negative_train_set))
print(len(negative_test_independent))

"""

In [ ]:
######## Train
"""
merge_train=np.zeros((len(positive_train_set)+len(negative_train_set),positive_train_set.shape[1]))
print(merge_train.shape)

merge_train=np.concatenate((positive_train_set,negative_train_set))
print(merge_train.shape)

##### label

merge_train_y=[]

for i in range(len(positive_train_set)):     #### for positive
    merge_train_y.append(1)

    
for j in range(len(negative_train_set)):    #### for negative
    merge_train_y.append(0)

print(len(merge_train_y))

"""

In [ ]:
######## Test
"""

merge_test=np.zeros((len(positive_test_independent)+len(negative_test_independent),positive_test_independent.shape[1]))
print(merge_test.shape)

merge_test=np.concatenate((positive_test_independent,negative_test_independent))
print(merge_test.shape)

##### label

merge_test_y=[]

for i in range(len(positive_test_independent)):     #### for positive
    merge_test_y.append(1)

    
for j in range(len(negative_test_independent)):    #### for negative
    merge_test_y.append(0)

print(len(merge_test_y))

"""

# To Solve Imbalance Problem With Central Based Undersampling

In [ ]:
print('Original dataset shape {}'.format(Counter(merge_train_y)))
cc = ClusterCentroids(random_state=42)
X_res, y_res = cc.fit_sample(merge_train, merge_train_y)
print('Resampled dataset shape {}'.format(Counter(y_res)))


# Data Split Into Test & Train

In [ ]:
x_train_set,x_test_independent,y_train_set,y_test_independent=train_test_split(X_res, y_res,test_size=0.2)
print("Test Size: ",len(x_test_independent))
print('In Test: {}'.format(Counter(y_test_independent)))
print("Train Size: ",len(x_train_set))
print('In Test: {}'.format(Counter(y_train_set)))

# Scaling Train & Test data

In [ ]:
"""
# define min max scaler
scaler = MinMaxScaler()

###### Train
# transform data
x_train_minmax = scaler.fit_transform(X_res)
#print(x_train_minmax)



##### Test
# transform data
x_test_minmax = scaler.fit_transform(merge_test)
#print(x_test_minmax)

"""

In [ ]:
###### Remove some errors in jupyter
"""
merge_y_test=np.array(merge_y_test)
print(merge_y_test.dtype)

y_train_set=np.array(y_res)
print(y_train_set.dtype)
"""

In [ ]:
###### Remove some errors in jupyter
"""
merge_y=np.array(merge_y)
print(merge_y.dtype)

merge_y_test=np.array(merge_y_test)
print(merge_y_test.dtype)
"""

# Support Vector Machine

In [ ]:
###### Remove some errors in jupyter

merge_y=np.array(merge_y)
print(merge_y.dtype)

merge_y_test=np.array(merge_y_test)
print(merge_y_test.dtype)

In [ ]:
################## KFold Cross Validation approach With NUS-2


clf = SVC(C=16)
kf = KFold(n_splits=10,shuffle=True)
kf.split(merge_feature_best)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
sn_model = []
sp_model = []
f1_model = []
mcc_model= []
auc_model= []


print("************************ Support Vector Machine ***********************")
 
# Iterate over each train-test split
for train_index, test_index in kf.split(merge_feature_best):
    
    # Split train-test
    X_train = merge_feature_best[train_index] 
    X_test = merge_feature_best[test_index]
    y_train = merge_y[train_index]
    y_test =  merge_y[test_index]
    
    # Train the model
    model = clf.fit(X_train, y_train)
    
    # Append to accuracy_model the accuracy of the model
    y_pred=model.predict(X_test)
    #y_pred_prob= model.predict_prob(X_test)
    accuracy_model.append(accuracy_score(y_test, y_pred))
    
    confusion=confusion_matrix(y_test,y_pred)
    print("Confusion Matrix: ",confusion)
    TP=confusion[1,1]
    print("TP:",TP)
    TN=confusion[0,0]
    print("TN: ",TN)
    FP=confusion[0,1]
    print("FP: ",FP)
    FN=confusion[1,0]
    print("FN: ",FN)
    
    SN=TP/(float)(TP+FN)
    sn_model.append(SN)
    print("Sensivity: ",SN)
    
    SP=TN/(float)(TN+FP)
    sp_model.append(SP)
    print("Specificity: ",SP)
    
    mcc= matthews_corrcoef(y_test,y_pred)
    mcc_model.append(mcc)
    print("MCC: ",mcc)
    
    a1=(TN+TP)/(float)(TN+TP+FN+FP)
    print("Accuracy from quation: ",a1)
    
    f1_model.append(f1_score(y_test, y_pred))
    
    auc=roc_auc_score(y_test,y_pred)
    auc_model.append(auc)
    print("AUC: ",auc)
    
    print("**************************************************")


print("************************ 10-Fold Cross Validation ***********************")

######Print the accuracy and others metrics 
print("Accuracy:",accuracy_model)
print("Sensitivity:",sn_model)
print("Specificity:",sp_model)
print("F-1 Score:",f1_model)
print("MCC Score:",mcc_model)
print("AUC Score:",auc_model)



a=m.fsum(accuracy_model)/len(accuracy_model)
print("Average Accuracy: ",a," ",np.std(accuracy_model))

b=m.fsum(sn_model)/len(accuracy_model)
print("Average Sensitivity: ",b," ",np.std(sn_model))

c=m.fsum(sp_model)/len(accuracy_model)
print("Average Specificity: ",c," ",np.std(sp_model))

d=m.fsum(f1_model)/len(accuracy_model)
print("Average f-1 Score: ",d)

e=m.fsum(mcc_model)/len(accuracy_model)
print("Average MCC Score: ",e," ",np.std(mcc_model))

f=m.fsum(auc_model)/len(auc_model)
print("Average AUC Score: ",f," ",np.std(auc_model))


print("************************ Independent Test Set ***********************")

# Train the model
#model = clf.fit(merge_feature_best, merge_y)

# Append to accuracy_model the accuracy of the model
y_pred=model.predict(f_test)

confusion=confusion_matrix(merge_y_test,y_pred)
print("Confusion Matrix: ",confusion)
TP=confusion[1,1]
print("TP:",TP)
TN=confusion[0,0]
print("TN: ",TN)
FP=confusion[0,1]
print("FP: ",FP)
FN=confusion[1,0]
print("FN: ",FN)

SN=TP/(float)(TP+FN)
sn_model.append(SN)
print("Sensivity: ",SN)
    
SP=TN/(float)(TN+FP)
sp_model.append(SP)
print("Specificity: ",SP)
    
mcc= matthews_corrcoef(merge_y_test,y_pred)
mcc_model.append(mcc)
print("MCC: ",mcc)
    
a1=(TN+TP)/(float)(TN+TP+FN+FP)
print("Accuracy from quation: ",a1)
    
auc=roc_auc_score(merge_y_test,y_pred)
auc_model.append(auc)
print("AUC: ",auc)
    
print("**************************************************")



# XG Boost

In [ ]:
################## KFold Cross Validation approach With Center based

from xgboost import XGBClassifier
import xgboost as xgb

clf = XGBClassifier()
kf = KFold(n_splits=10,shuffle=True)
kf.split(merge_feature_best)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
sn_model = []
sp_model = []
f1_model = []
mcc_model= []
auc_model= []


print("************************ XG Boost ***********************")
 
# Iterate over each train-test split
for train_index, test_index in kf.split(merge_feature_best):
    
    # Split train-test
    X_train = merge_feature_best[train_index] 
    X_test = merge_feature_best[test_index]
    y_train = merge_y[train_index]
    y_test =  merge_y[test_index]
    
    # Train the model
    model = clf.fit(X_train, y_train)
    
    # Append to accuracy_model the accuracy of the model
    y_pred=model.predict(X_test)
    #y_pred_prob= model.predict_prob(X_test)
    accuracy_model.append(accuracy_score(y_test, y_pred)*100)
    
    confusion=confusion_matrix(y_test,y_pred)
    print("Confusion Matrix: ",confusion)
    TP=confusion[1,1]
    print("TP:",TP)
    TN=confusion[0,0]
    print("TN: ",TN)
    FP=confusion[0,1]
    print("FP: ",FP)
    FN=confusion[1,0]
    print("FN: ",FN)
    
    SN=TP/(float)(TP+FN)
    sn_model.append(SN)
    print("Sensivity: ",SN)
    
    SP=TN/(float)(TN+FP)
    sp_model.append(SP)
    print("Specificity: ",SP)
    
    mcc= matthews_corrcoef(y_test,y_pred)
    mcc_model.append(mcc)
    print("MCC: ",mcc)
    
    a1=(TN+TP)/(float)(TN+TP+FN+FP)
    print("Accuracy from quation: ",a1)
    
    f1_model.append(f1_score(y_test, y_pred))
    
    auc=roc_auc_score(y_test,y_pred)
    auc_model.append(auc)
    print("AUC: ",auc)
    
    print("**************************************************")


print("************************ 10-Fold Cross Validation ***********************")

######Print the accuracy and others metrics 
print("Accuracy:",accuracy_model)
print("Sensitivity:",sn_model)
print("Specificity:",sp_model)
print("F-1 Score:",f1_model)
print("MCC Score:",mcc_model)
print("AUC Score:",auc_model)



a=sum(accuracy_model)/len(accuracy_model)
print("Average Accuracy: ",a," ",np.std(accuracy_model))

b=sum(sn_model)/len(accuracy_model)
print("Average Sensitivity: ",b," ",np.std(sn_model))

c=sum(sp_model)/len(accuracy_model)
print("Average Specificity: ",c," ",np.std(sp_model))

d=sum(f1_model)/len(accuracy_model)
print("Average f-1 Score: ",d)

e=sum(mcc_model)/len(accuracy_model)
print("Average MCC Score: ",e," ",np.std(mcc_model))

f=sum(auc_model)/len(auc_model)
print("Average AUC Score: ",f," ",np.std(auc_model))



print("************************ Independent Test Set ***********************")

# Train the model
#model = clf.fit(x_train_minmax, y_train_set)

# Append to accuracy_model the accuracy of the model
y_pred=model.predict(f_test)


confusion=confusion_matrix(merge_y_test,y_pred)
print("Confusion Matrix: ",confusion)
TP=confusion[1,1]
print("TP:",TP)
TN=confusion[0,0]
print("TN: ",TN)
FP=confusion[0,1]
print("FP: ",FP)
FN=confusion[1,0]
print("FN: ",FN)

SN=TP/(float)(TP+FN)
sn_model.append(SN)
print("Sensivity: ",SN)
    
SP=TN/(float)(TN+FP)
sp_model.append(SP)
print("Specificity: ",SP)
    
mcc= matthews_corrcoef(merge_y_test,y_pred)
mcc_model.append(mcc)
print("MCC: ",mcc)
    
a1=(TN+TP)/(float)(TN+TP+FN+FP)
print("Accuracy from quation: ",a1)
    
    
auc=roc_auc_score(merge_y_test,y_pred)
auc_model.append(auc)
print("AUC: ",auc)
    
print("**************************************************")



# Random Forest 

In [ ]:
################## KFold Cross Validation approach With Center based

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
kf = KFold(n_splits=10,shuffle=True)
kf.split(merge_feature_best)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
sn_model = []
sp_model = []
f1_model = []
mcc_model= []
auc_model= []

 
print("************************ Random Forest ***********************")    
    
# Iterate over each train-test split
for train_index, test_index in kf.split(merge_feature_best):
    
    # Split train-test
    X_train = merge_feature_best[train_index] 
    X_test = merge_feature_best[test_index]
    y_train = merge_y[train_index]
    y_test =  merge_y[test_index]
    
    # Train the model
    model = clf.fit(X_train, y_train)
    
    # Append to accuracy_model the accuracy of the model
    y_pred=model.predict(X_test)
    #y_pred_prob= model.predict_prob(X_test)
    accuracy_model.append(accuracy_score(y_test, y_pred)*100)
    
    confusion=confusion_matrix(y_test,y_pred)
    print("Confusion Matrix: ",confusion)
    TP=confusion[1,1]
    print("TP:",TP)
    TN=confusion[0,0]
    print("TN: ",TN)
    FP=confusion[0,1]
    print("FP: ",FP)
    FN=confusion[1,0]
    print("FN: ",FN)
    
    SN=TP/(float)(TP+FN)
    sn_model.append(SN)
    print("Sensivity: ",SN)
    
    SP=TN/(float)(TN+FP)
    sp_model.append(SP)
    print("Specificity: ",SP)
    
    mcc= matthews_corrcoef(y_test,y_pred)
    mcc_model.append(mcc)
    print("MCC: ",mcc)
    
    a1=(TN+TP)/(float)(TN+TP+FN+FP)
    print("Accuracy from quation: ",a1)
    
    f1_model.append(f1_score(y_test, y_pred))
    
    auc=roc_auc_score(y_test,y_pred)
    auc_model.append(auc)
    print("AUC: ",auc)
    
    print("**************************************************")


print("************************ 10-Fold Cross Validation ***********************")

######Print the accuracy and others metrics 
print("Accuracy:",accuracy_model)
print("Sensitivity:",sn_model)
print("Specificity:",sp_model)
print("F-1 Score:",f1_model)
print("MCC Score:",mcc_model)
print("AUC Score:",auc_model)



a=sum(accuracy_model)/len(accuracy_model)
print("Average Accuracy: ",a," ",np.std(accuracy_model))

b=sum(sn_model)/len(accuracy_model)
print("Average Sensitivity: ",b," ",np.std(sn_model))

c=sum(sp_model)/len(accuracy_model)
print("Average Specificity: ",c," ",np.std(sp_model))

d=sum(f1_model)/len(accuracy_model)
print("Average f-1 Score: ",d)

e=sum(mcc_model)/len(accuracy_model)
print("Average MCC Score: ",e," ",np.std(mcc_model))

f=sum(auc_model)/len(auc_model)
print("Average AUC Score: ",f," ",np.std(auc_model))



print("************************ Independent Test Set ***********************")

# Train the model
#model = clf.fit(x_train_minmax, y_train_set)

# Append to accuracy_model the accuracy of the model
y_pred=model.predict(f_test)

confusion=confusion_matrix(merge_y_test,y_pred)
print("Confusion Matrix: ",confusion)
TP=confusion[1,1]
print("TP:",TP)
TN=confusion[0,0]
print("TN: ",TN)
FP=confusion[0,1]
print("FP: ",FP)
FN=confusion[1,0]
print("FN: ",FN)

SN=TP/(float)(TP+FN)
sn_model.append(SN)
print("Sensivity: ",SN)
    
SP=TN/(float)(TN+FP)
sp_model.append(SP)
print("Specificity: ",SP)
    
mcc= matthews_corrcoef(merge_y_test,y_pred)
mcc_model.append(mcc)
print("MCC: ",mcc)
    
a1=(TN+TP)/(float)(TN+TP+FN+FP)
print("Accuracy from quation: ",a1)

  
auc=roc_auc_score(merge_y_test,y_pred)
auc_model.append(auc)
print("AUC: ",auc)
    
print("**************************************************")



# Light GBM

In [ ]:
################## KFold Cross Validation approach With Center based

import lightgbm as lgb

#Specifying the parameter
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='binary' #Binary target feature
params['metric']='binary_logloss' #metric for binary classification
params['max_depth']=10


kf = KFold(n_splits=10,shuffle=True)
kf.split(merge_feature_best)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
sn_model = []
sp_model = []
f1_model = []
mcc_model= []
auc_model= []


print("********************** Light GBM************************")
 
# Iterate over each train-test split
for train_index, test_index in kf.split(merge_feature_best):
    
    # Split train-test
    X_train = merge_feature_best[train_index] 
    X_test = merge_feature_best[test_index]
    y_train = merge_y[train_index]
    y_test =  merge_y[test_index]
    
    #converting the dataset into proper LGB format 
    d_train=lgb.Dataset(X_train, label=y_train)

    #train the model 
    clf=lgb.train(params,d_train)
    
    # Append to accuracy_model the accuracy of the model
    y_pred=model.predict(X_test)
    accuracy_model.append(accuracy_score(y_test, y_pred)*100)
    
    confusion=confusion_matrix(y_test,y_pred)
    print("Confusion Matrix: ",confusion)
    TP=confusion[1,1]
    print("TP:",TP)
    TN=confusion[0,0]
    print("TN: ",TN)
    FP=confusion[0,1]
    print("FP: ",FP)
    FN=confusion[1,0]
    print("FN: ",FN)
    
    SN=TP/(float)(TP+FN)
    sn_model.append(SN)
    print("Sensivity: ",SN)
    
    SP=TN/(float)(TN+FP)
    sp_model.append(SP)
    print("Specificity: ",SP)
    
    mcc= matthews_corrcoef(y_test,y_pred)
    mcc_model.append(mcc)
    print("MCC: ",mcc)
    
    a1=(TN+TP)/(float)(TN+TP+FN+FP)
    print("Accuracy from quation: ",a1)
    
    f1_model.append(f1_score(y_test, y_pred))
    
    auc=roc_auc_score(y_test,y_pred)
    auc_model.append(auc)
    print("AUC: ",auc)
    
    print("**************************************************")


print("************************ 10-Fold Cross Validation ***********************")

######Print the accuracy and others metrics 
print("Accuracy:",accuracy_model)
print("Sensitivity:",sn_model)
print("Specificity:",sp_model)
print("F-1 Score:",f1_model)
print("MCC Score:",mcc_model)
print("AUC Score:",auc_model)



a=sum(accuracy_model)/len(accuracy_model)
print("Average Accuracy: ",a," ",np.std(accuracy_model))

b=sum(sn_model)/len(accuracy_model)
print("Average Sensitivity: ",b," ",np.std(sn_model))

c=sum(sp_model)/len(accuracy_model)
print("Average Specificity: ",c," ",np.std(sp_model))

d=sum(f1_model)/len(accuracy_model)
print("Average f-1 Score: ",d)

e=sum(mcc_model)/len(accuracy_model)
print("Average MCC Score: ",e," ",np.std(mcc_model))

f=sum(auc_model)/len(auc_model)
print("Average AUC Score: ",f," ",np.std(auc_model))



print("************************ Independent Test Set ***********************")

#converting the dataset into proper LGB format 
d_train=lgb.Dataset(merge_feature_best, label=merge_y)

#Specifying the parameter
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt'       #GradientBoostingDecisionTree
params['objective']='binary'         #Binary target feature
params['metric']='binary_logloss'    #metric for binary classification
params['max_depth']=10


#train the model 
clf=lgb.train(params,d_train)

# Append to accuracy_model the accuracy of the model
y_pred=model.predict(f_test)

confusion=confusion_matrix(merge_y_test,y_pred)
print("Confusion Matrix: ",confusion)
TP=confusion[1,1]
print("TP:",TP)
TN=confusion[0,0]
print("TN: ",TN)
FP=confusion[0,1]
print("FP: ",FP)
FN=confusion[1,0]
print("FN: ",FN)

SN=TP/(float)(TP+FN)
sn_model.append(SN)
print("Sensivity: ",SN)
    
SP=TN/(float)(TN+FP)
sp_model.append(SP)
print("Specificity: ",SP)
    
mcc= matthews_corrcoef(merge_y_test,y_pred)
mcc_model.append(mcc)
print("MCC: ",mcc)
    
a1=(TN+TP)/(float)(TN+TP+FN+FP)
print("Accuracy from quation: ",a1)
    
auc=roc_auc_score(merge_y_test,y_pred)
auc_model.append(auc)
print("AUC: ",auc)
    
print("**************************************************")

